# Introduction to Opik Agent Optimizer

You will need:

1. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
2. An OpenAI account, for using an LLM
[platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)


## Setup

This pip-install takes about a minute.

In [ ]:
%%capture
%pip install opik-optimizer

Let's make sure we have the correct version:

In [1]:
import opik_optimizer

opik_optimizer.__version__

'0.7.3'

The version should be 0.7.3 or greater.

[Comet](https://www.comet.com/site?from=llm&utm_source=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=langchain&utm_campaign=opik) and grab your API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik) for more information.

Enter your Comet API key, followed by "Y" to use your own workspace:

In [ ]:
import opik

# Configure Opik
opik.configure()

For this example, we'll use OpenAI models, so we need to set our OpenAI API key:

In [ ]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

To optimize any prompt, we'll need three basic things:

1. A starting prompt
2. A metric
3. A dataset

## The Dataset

In this experiment, we are going to use the **HotPotQA** dataset. This dataset was designed to be difficult for regular LLMs to handle. This dataset is called a "**multi-hop**" dataset because answering the questions involves multiple reasoning steps and multiple tool calls, where the LLM needs to infer relationships, combine information, or draw conclusions based on the combined context.

Example:

> "What are the capitals of the states that border California?"

You'd need to find which states border California, and then lookup each state's capital.

The dataset has about 113,000 such crowd-sourced questions that are constructed to require the introductory paragraphs of two Wikipedia articles to answer.

**NOTE:** The name "HotPot" comes from the restaurant where the authors came up with the idea of the dataset.

In [2]:
from opik_optimizer.demo import get_or_create_dataset

opik_dataset = get_or_create_dataset("hotpot-300")

Let's take a look at some dataset items:

In [3]:
rows = opik_dataset.get_items()
rows[0]

{'id': '0195d400-517b-7f17-b746-ff3d084463ac',
 'question': 'Were both drinks, the Smoking Bishop and the Mickey Slim, popular in different countries?',
 'answer': 'yes'}

We see that each item has a "question" and "answer". Some of the answers are short and direct, and some of them are more complicated:

In [5]:
rows[1]

{'id': '0195d400-517a-70d2-9e9e-952765e7b829',
 'question': 'The 6th Canadian Infantry Division was originally scheduled to participate in this proposed Allied plan for the invasion of Japan near the end of World War II but the plan was abandoned after the Soviet invasion of which region? ',
 'answer': 'Manchuria'}

## Opik Project

All LLM traces in Opik are saved in a "project". We'll put them all in the following project name:

In [8]:
project_name = "optimize-workshop-2025"

## The Metric

Choosing a good metric for optimization is tricky. For these examples, we'll pick one that will allow us to show improvement, and also provide a gradient of scores. In general though, this metric isn't the best for optimization runs.

We'll use "Edit Distance" AKA "Levenshtein Distance":

In [9]:
from opik.evaluation.metrics import LevenshteinRatio

metric = LevenshteinRatio(project_name=project_name)

The metric takes two things: the `output` of the LLM and the `reference` (the truth):

In [10]:
metric.score("Hello", "Hello")

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0196ca26-6722-75ae-8431-edc12f49aefb&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='levenshtein_ratio_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

In [11]:
metric.score("Hello!", "Hello")

ScoreResult(name='levenshtein_ratio_metric', value=0.9090909090909091, reason=None, metadata=None, scoring_failed=False)

The edit distance between "Hello!" and "Hello" is 1. Here is how the .91 is computed:

In [12]:
edit_distance = 1

1 - edit_distance / (len("Hello1") + len("Hello"))

0.9090909090909091

For more information see: [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

## Configuation

To create the necesary configurations for using an Opik Agent Optimizer, you'll need three things:

1. An initial prompt
2. A MetricConfig
3. A TaskConfig

We're going to start with a pretty bad prompt... so we can optimize it!

In [13]:
initial_prompt = "Provide an answer to the question"

The the two configurations:

In [14]:
from opik_optimizer import (
    MetricConfig,
    TaskConfig,
    from_llm_response_text,
    from_dataset_field,
)

metric_config = MetricConfig(
    metric=LevenshteinRatio(project_name=project_name),
    inputs={
        "output": from_llm_response_text(),
        "reference": from_dataset_field(name="answer"),
    },
)

task_config = TaskConfig(
    instruction_prompt=initial_prompt,
    input_dataset_fields=["question"],
    output_dataset_field="answer",
    use_chat_prompt=True,
)

As you can see the `MetricConfig` is composed of our chosen metric. In addition, we need to know what the inputs will be. The inputs here are actually the outputs from the LLM.

We need two inputs for the metric:
1. The output produced by the LLM (uses a special name)
2. The correct answer (provided by the database item "answer")

The `TaskConfig` defines how to process a prompt. We need the initial prompt, and the inputs and outputs of the dataset.

In this case, we will use the chat_prompt format as our result.

## FewShotBayesianOptimizer

The FewShotBayesianOptimizer name indicates two things:

1. It will produce Chat Prompts, or FewShot examples as described in the slides.
2. Secondly, it describes how it searches for the best set of these FewShot examples.

To use this optimizer, we import it and create an instance, passing in the project name and model parameters:

In [15]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
)

optimizer = FewShotBayesianOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

### Baseline

Before we optimize this prompt ("Provide an answer to the question") let's see what the bare prompt does by itself on the dataset:

In [16]:
score = optimizer.evaluate_prompt(
    dataset=opik_dataset,
    metric_config=metric_config,
    task_config=task_config,
    prompt=initial_prompt,
    n_samples=100,
)
score

Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ hotpot-300 (100 samples) ─────────────╮
│                                        │
│ Total time:        00:00:11            │
│ Number of samples: 100                 │
│                                        │
│ levenshtein_ratio_metric: 0.1672 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=752475;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196ca28-0fa9-7f56-b445-5a8321962455&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

0.1671873373886302

In my run, it scored about 16% correct. [I say "percent correct" but because we are using edit distance, that isn't quite accurate. But we can think of it this way.] Your result may be somewhat different but is probably between 10% and 20% correct.

Ok, let's optimize that prompt!

In [17]:
result = optimizer.optimize_prompt(
    opik_dataset, metric_config, task_config, n_trials=3, n_samples=50
)

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

╭─ hotpot-300 (50 samples) ──────────────╮
│                                        │
│ Total time:        00:00:04            │
│ Number of samples: 50                  │
│                                        │
│ levenshtein_ratio_metric: 0.1697 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=475435;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196ca29-6305-722a-b8cc-478f877977a9&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

...

View the results ]8;id=372528;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196ca29-90a9-77c9-960f-4c2b09adb242&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

Evaluation:   0%|          | 0/50 [00:00<?, ?it/s]

╭─ hotpot-300 (50 samples) ──────────────╮
│                                        │
│ Total time:        00:00:05            │
│ Number of samples: 50                  │
│                                        │
│ levenshtein_ratio_metric: 0.5009 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=279946;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=0196ca29-aa06-7da0-90d7-762dcd58617a&dataset_id=0195d400-52bc-7ac1-ae8e-74931512157f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

In [18]:
result.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         Optimizer                                                                                    ║
║ Model Used:        openai/gpt-4o-mini (Temp: 0.1)                                                               ║
║ Metric Evaluated:  levenshtein_ratio_metric                                                                     ║
║ Initial Score:     0.1697                                                                                       ║
║ Final Best Score:  0.5009                                                                                       ║
║ Total Improvement: 195.20%                                                                                      ║
║ Rounds Completed:  0                                                                                            ║
║ Stopped Early:     False                                                                                        ║
║                                                                                                                 ║
║                                                                                                                 ║
║ ╭────────────────────────────────── Final Optimized Prompt (Chat Structure) ──────────────────────────────────╮ ║
║ │                                                                                                             │ ║
║ │  Instruction: Provide an answer to the question                                                             │ ║
║ │  ---                                                                                                        │ ║
║ │  System: Provide an answer to the question                                                                  │ ║
║ │                                                                                                             │ ║
║ │  You are an intelligent assistant that learns from few-shot examples provided earlier in the conversation.  │ ║
║ │  Whenever you respond, carefully follow the structure, tone, and format of previous assistant replies,      │ ║
║ │  using them as a guide                                                                                      │ ║
║ │  ---                                                                                                        │ ║
║ │  User:                                                                                                      │ ║
║ │  {                                                                                                          │ ║
║ │    "question": "What precious metal did both McIntyre Mines and the Kemess Mine dig for?"                   │ ║
║ │  }                                                                                                          │ ║
║ │                                                                                                             │ ║
║ │  ---                                                                                                        │ ║
║ │  Assistant: gold                                                                                            │ ║
║ │  ---                                                                                                        │ ║
║ │  User:                                                                                                      │ ║
║ │  {                                                                                                          │ ║
║ │    "question": "Which member of a collegiate athletic conference comprising sports teams from eight         │ ║
║ │  private institutions of higher education in the Northeastern United States played their home games at      │ ║
║ │  Robert K. Kraft Field at Lawrence A. Wien Stadium?"

Well done optimizer!

The percentage correct went from about 15% to about 50% correct.

What did we find? The result is a series of messages:

In [19]:
result.details["chat_messages"]

[{'role': 'system',
  'content': 'Provide an answer to the question\n\nYou are an intelligent assistant that learns from few-shot examples provided earlier in the conversation. Whenever you respond, carefully follow the structure, tone, and format of previous assistant replies, using them as a guide'},
 {'role': 'user',
  'content': '\n{\n  "question": "What precious metal did both McIntyre Mines and the Kemess Mine dig for?"\n}\n'},
 {'role': 'assistant', 'content': 'gold'},
 {'role': 'user',
  'content': '\n{\n  "question": "Which member of a collegiate athletic conference comprising sports teams from eight private institutions of higher education in the Northeastern United States played their home games at Robert K. Kraft Field at Lawrence A. Wien Stadium?"\n}\n'},
 {'role': 'assistant', 'content': 'The 2011 Columbia Lions'},
 {'role': 'user',
  'content': '\n{\n  "question": "Which popular magazine was published first, Inside Sports or Men\'s Fitness?"\n}\n'},
 {'role': 'assistant'

## Opik Visualization UI

When you create an Optimization Run, you'll see it in the Opik UI (either running locally or hosted).

If you go to your `comet.com/opik/YOURID/optimizations` page, you'll see your run at the top of the list:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimizer-ui.png" width="500px">

Along the top of the page you'll see a running history of the metric scores, with the latest dataset selected. 

If you click on your run, you'll see the set of trials that ran durring this optimization run. Running across the top of this page are the scores just for this trial. On the top right you'll see the best so far:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimize-trials.png" width="500px">

If you click on a trial, you'll see the prompt for that trial:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimizer-prompt.png" width="500px">

From the trial page you can also see the Trial items, and even dig down (mouse over the "Evaluation task" column on a row) to see the traces.

## Using Optimized Prompts

How can we use the optimized results?


Once we have the "chat_messages", we can do the following:

In [20]:
from litellm.integrations.opik.opik import OpikLogger
import litellm

opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]


def query(question, chat_messages):
    messages = chat_messages[:-1]  # Cut off the last one
    # replace it with question in proper format:
    messages.append({"role": "user", "content": '{"question": "%s"}"}' % question})

    response = litellm.completion(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=5000,
        messages=messages,
    )
    return response.choices[0].message.content

In [21]:
query("When was David Chalmers born?", result.details["chat_messages"])

'David Chalmers was born on April 20, 1966.'

In [22]:
query("What weighs more: a baby elephant or an SUV?", result.details["chat_messages"])

'A baby elephant typically weighs more than an SUV.'

If it says "elephant" that is not correct!

We'll need to use an agent with tools to get a better answer.

# Next Steps

You can try out other optimizers. More details can be found in the [Opik Agent Optimizer documentation](https://www.comet.com/docs/opik/agent_optimization/overview).